# Data Cleaning and Preprocessing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# 1. reading raw data
raw_data = pd.read_csv("./housing_data/train.csv")

In [3]:
# 2. selecting useful features and drop columns with nan
filtered_features = ["MSZoning", "LotFrontage", "LotArea", "Street", "LotShape", "LandContour", "Utilities", "LotConfig", "LandSlope", "BldgType", "HouseStyle", 
                     "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", "MasVnrArea", "ExterQual", "ExterCond", "Foundation", "BsmtQual", "BsmtCond", 
                     "BsmtExposure", "BsmtUnfSF", "TotalBsmtSF", "Heating", "HeatingQC", "CentralAir", "1stFlrSF", "2ndFlrSF", "GrLivArea", "BsmtFullBath", "BsmtHalfBath", 
                     "FullBath", "HalfBath", "KitchenAbvGr", "KitchenQual", "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu", "GarageCars", "GarageArea", 
                     "GarageQual", "GarageCond", "PavedDrive", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch", "PoolArea", "PoolQC", "Fence", 
                     "MiscVal", "YrSold", "SaleType", "SaleCondition"]
filtered_data = raw_data[filtered_features]

In [4]:
# 3. encoding ordinal categorical variables
ordinal_columns = ["Utilities", "LandSlope", "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "BsmtExposure",
                   "HeatingQC", "KitchenQual", "Functional", "FireplaceQu", "GarageQual", "GarageCond", "PavedDrive", 
                   "PoolQC", "Fence"]

# deal with meaningful NaN
filtered_data = filtered_data.copy()
meaningful_nan = 'not_applicable'
filtered_data[ordinal_columns] = filtered_data[ordinal_columns].fillna(meaningful_nan)

ordinal_order = {
    "Utilities": ["ELO", "NoSeWa", "NoSewr", "AllPub"],
    "LandSlope": ["Sev", "Mod", "Gtl"],
    "ExterQual": ["Po", "Fa", "TA", "Gd", "Ex"],
    "ExterCond": ["Po", "Fa", "TA", "Gd", "Ex"],
    "BsmtQual": ["Po", "Fa", meaningful_nan, "TA", "Gd", "Ex"],
    "BsmtCond": ["Po", "Fa", meaningful_nan, "TA", "Gd", "Ex"],
    "BsmtExposure": ["No", "Mn", meaningful_nan, "Av", "Gd"],
    "HeatingQC": ["Po", "Fa", "TA", "Gd", "Ex"],
    "KitchenQual": ["Po", "Fa", "TA", "Gd", "Ex"],
    "Functional": ["Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"],
    "FireplaceQu": ["Po", "Fa", meaningful_nan, "TA", "Gd", "Ex"],
    "GarageQual": ["Po", "Fa", meaningful_nan, "TA", "Gd", "Ex"],
    "GarageCond": ["Po", "Fa", meaningful_nan, "TA", "Gd", "Ex"],
    "PavedDrive": ["N", "P", "Y"],
    "PoolQC": ["Fa", "TA", meaningful_nan, "Gd", "Ex"],
    "Fence": ["MnWw", "MnPrv", meaningful_nan, "GdWo", "GdPrv"]
}
# ordinal encoding
for col in ordinal_columns:
    ordinal_encoder = OrdinalEncoder(categories=[ordinal_order[col]])
    ordinal_data = ordinal_encoder.fit_transform(filtered_data[[col]])
    filtered_data.loc[:, col] = ordinal_data

In [5]:
# 4. encoding nominal categorical variable
nominal_columns = ["MSZoning", "Street", "LotShape", "LandContour", "LotConfig", "BldgType", "HouseStyle", 
                   "Foundation", "Heating", "CentralAir", "SaleType", "SaleCondition"]
# one-hot encoding
nominal_data = pd.get_dummies(filtered_data[nominal_columns])
# remove original nominal data and concat the one-hot-encoded data
encoded_data = pd.concat([filtered_data.drop(nominal_columns, axis=1), nominal_data], axis=1)

In [6]:
# 5. feature engineering
encoded_data["year_since_built"] = encoded_data["YrSold"] - encoded_data["YearBuilt"]
encoded_data["year_since_remod"] = encoded_data["YrSold"] - encoded_data["YearRemodAdd"]
encoded_data.drop(["YrSold", "YearBuilt", "YearRemodAdd"], axis = 1, inplace = True)

In [7]:
# 6. normalization
scaler = StandardScaler()
encoded_data_scaled = scaler.fit_transform(encoded_data)
encoded_data_scaled = pd.DataFrame(encoded_data_scaled, columns=encoded_data.columns)

In [8]:
# 7. dealing NaN for for numeric variables
numerical_columns = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
                     '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
                     'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', "year_since_built", "year_since_remod"]
for col in numerical_columns:
    encoded_data_scaled[col].fillna(encoded_data_scaled[col].mean(), inplace=True)
encoded_data_scaled[numerical_columns].isnull().sum()

LotFrontage         0
LotArea             0
OverallQual         0
OverallCond         0
MasVnrArea          0
BsmtUnfSF           0
TotalBsmtSF         0
1stFlrSF            0
2ndFlrSF            0
GrLivArea           0
BsmtFullBath        0
BsmtHalfBath        0
FullBath            0
HalfBath            0
KitchenAbvGr        0
TotRmsAbvGrd        0
Fireplaces          0
GarageCars          0
GarageArea          0
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
MiscVal             0
year_since_built    0
year_since_remod    0
dtype: int64

In [12]:
# train validation test split
# 0.6 : 0.2 : 0.2
X_train, X_test, y_train, y_test = train_test_split(encoded_data_scaled, raw_data["SalePrice"], test_size=0.20, random_state=4244997347)

In [13]:
# Export train, test, and validation dataset
X_train.to_csv('./data/X_train.csv', index = False)
y_train.to_csv('./data/y_train.csv', index = False)
X_test.to_csv('./data/X_test.csv', index = False)
y_test.to_csv('./data/y_test.csv', index = False)